# Dataset Preparation

## Setup

In [1]:
pwd()

'/home/hnasrulloh/Works/pangenomes/notebooks'

In [2]:
from collections import defaultdict, namedtuple
from os import path, listdir, makedirs
import re

import pandas as pd
import numpy as np

In [3]:
DATA_DIR = path.join(path.pardir, 'data')

PATRIC_AMR = path.join(DATA_DIR, 'PATRIC_genomes_AMR.txt')
GENOME_SUMMARY = path.join(DATA_DIR, 'genome_summary')
GENOME_METADATA = path.join(DATA_DIR, 'genome_metadata')

SAMPLES_DIR = path.join(DATA_DIR, 'samples')

## 1. Samples

### 1.1 Genomes selection

In [4]:
dtypes = {
    'genome_id': str, 
    'genome_name': str,
    # 'taxon_id' 'genome_status'
    'genome_length': np.float64,
    'gc_content': np.float64,
    # 'contig_l50' 'contig_n50' 'chromosomes' 'plasmids' 'contigs'
    # 'patric_cds' 'refseq_cds' 'trna' 'rrnacoarse_consistency'
    'fine_consistency': np.float64,
    'checkm_completeness': np.float64,
    'checkm_contamination': np.float64,
    # 'genome_qualitydate_created' 'date_modified'
}

df = pd.read_csv(
    GENOME_SUMMARY, 
    sep='\t',
    usecols=dtypes.keys(),
    dtype=dtypes
)

In [5]:
def pipe(df, funcs):
    for f in funcs:
        df = f(df)
    return df


def f_mtb(df):
    return df[df.genome_name.str.contains('Mycobacterium tuberculosis')]


def f_gc_content(df):
    # https://bionumbers.hms.harvard.edu/bionumber.aspx?id=101990
    value = 65.6
    lower_bound = value - 0.1
    upper_bound = value + 0.1
    
    return df[(df.gc_content >= lower_bound) & (df.gc_content <= upper_bound)]


def f_genome_length(df):
    # https://bionumbers.hms.harvard.edu/bionumber.aspx?id=101741
    value = 4.41e6
    lower_bound = value - 0.1e6
    upper_bound = value + 0.1e6
    
    return df[(df.genome_length >= lower_bound) & (df.genome_length <= upper_bound)]


def f_qc(df):
    # https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0250092
    # Use more strict QC policy
    contamination_threshold = 10 - 9
    completeness_threshold = 80 + 19
    consistency_threshold = 87 + 11
    
    df = df[df.checkm_contamination <= contamination_threshold]
    df = df[df.checkm_completeness >= completeness_threshold]
    df = df[df.fine_consistency >= consistency_threshold]
    return df


df = pipe(df, [
    f_mtb, 
    f_gc_content, 
    f_genome_length,
    f_qc,
])
df

,genome_id,genome_name,genome_length,gc_content,fine_consistency,checkm_completeness,checkm_contamination
315612,1733.752,Mycobacterium tuberculosis G-032_S_1,4322020.0,65.55000,98.4,100.0,0.0
315613,1733.749,Mycobacterium tuberculosis G-036_S_2,4327952.0,65.55000,98.4,100.0,0.0
315614,1733.764,Mycobacterium tuberculosis G-038_S-1,4328089.0,65.54000,98.3,100.0,0.0
315615,1733.797,Mycobacterium tuberculosis G-039-C,4324832.0,65.54000,98.6,100.0,0.0
315616,1733.815,Mycobacterium tuberculosis G-039-E,4331941.0,65.52000,98.2,100.0,0.0
...,...,...,...,...,...,...,...
346926,1773.2434,Mycobacterium tuberculosis strain UM-CSF15,4374121.0,65.61000,99.2,100.0,0.0
346927,1773.2432,Mycobacterium tuberculosis strain UM-CSF17,4356783.0,65.58000,99.0,100.0,0.0
346944,1773.10508,Mycobacterium tuberculosis strain UT1,4357245.0,65.61882,99.4,100.0,0.0
346967,1773.213,Mycobacterium tuberculosis strain VRFCWCF XDRT...,4369955.0,65.50000,99.5,100.0,0.0


### 1.2 Genomes metadata enrichment

In [6]:
dtypes = {
    'genome_id': str,
    # 'genome_name' 'organism_name' 
    'taxon_id': str, 
    # 'genome_status'
    'strain': str,
    # 'serovar' 'biovar' 'pathovar' 'mlst' 'other_typing' 'culture_collection' 'type_strain'
    'completion_date': str,
    'publication': str,
    'bioproject_accession': str,
    'biosample_accession': str,
    'assembly_accession': str,
    'genbank_accessions': str,
    'refseq_accessions': str,
    'sequencing_centers': str,
    'sequencing_status': str,
    'sequencing_platform': str,
    'sequencing_depth': str,
    'assembly_method': str,
    'chromosomes': str,
    # 'plasmids'
    'contigs': np.float64,
    'sequences': np.float64,
    # 'genome_length' 'gc_content'
    'patric_cds': np.float64,
    # 'brc1_cds'
    'refseq_cds': np.float64,
    'isolation_site': str,
    'isolation_source': str,
    'isolation_comments': str,
    'collection_date': str,
    # 'isolation_country'
    'geographic_location': str,
    # 'latitude' 'longitude' 'altitude' 'depth' 'other_environmental'
    'host_name': str,
    # 'host_gender' 'host_age' 'host_health' 'body_sample_site' 'body_sample_subsite' 'other_clinical'
    # 'antimicrobial_resistance' 'antimicrobial_resistance_evidence'
    # 'gram_stain' 'cell_shape' 'motility' 'sporulation' 'temperature_range' 'optimal_temperature'
    # 'salinity' 'oxygen_requirement' 'habitat' 'disease'
    'comments': str,
    # 'additional_metadata'
}

df_metadata = pd.read_csv(
    GENOME_METADATA, 
    sep='\t',
    usecols=dtypes.keys(),
    dtype=dtypes
)

df = pd.merge(df, df_metadata, how='inner', on='genome_id')
df

,genome_id,genome_name,genome_length,gc_content,fine_consistency,checkm_completeness,checkm_contamination,taxon_id,strain,completion_date,...,sequences,patric_cds,refseq_cds,isolation_site,isolation_source,isolation_comments,collection_date,geographic_location,host_name,comments
0,1733.752,Mycobacterium tuberculosis G-032_S_1,4322020.0,65.55000,98.4,100.0,0.0,1773,G-032_S_1,NaN,...,NaN,4313.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1733.749,Mycobacterium tuberculosis G-036_S_2,4327952.0,65.55000,98.4,100.0,0.0,1773,G-036_S_2,NaN,...,NaN,4313.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1733.764,Mycobacterium tuberculosis G-038_S-1,4328089.0,65.54000,98.3,100.0,0.0,1773,G-038_S-1,NaN,...,NaN,4318.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1733.797,Mycobacterium tuberculosis G-039-C,4324832.0,65.54000,98.6,100.0,0.0,1773,G-039-C,NaN,...,NaN,4311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1733.815,Mycobacterium tuberculosis G-039-E,4331941.0,65.52000,98.2,100.0,0.0,1773,G-039-E,NaN,...,NaN,4318.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5901,1773.2434,Mycobacterium tuberculosis strain UM-CSF15,4374121.0,65.61000,99.2,100.0,0.0,1773,UM-CSF15,2016-05-09T00:00:00Z,...,NaN,4321.0,NaN,NaN,cerebrospinal fluid,NaN,NaN,NaN,"Human, Homo sapiens",Genomic differences between pulmonary and meni...
5902,1773.2432,Mycobacterium tuberculosis strain UM-CSF17,4356783.0,65.58000,99.0,100.0,0.0,1773,UM-CSF17,2016-05-09T00:00:00Z,...,NaN,4308.0,NaN,NaN,cerebrospinal fluid,NaN,NaN,NaN,"Human, Homo sapiens",Genomic differences between pulmonary and meni...
5903,1773.10508,Mycobacterium tuberculosis strain UT1,4357245.0,65.61882,99.4,100.0,0.0,1773,UT1,2018-07-18T00:00:00Z,...,NaN,4326.0,4063.0,NaN,Sputum,NaN,2005-06,Colombia:Medellin,"Human, Homo sapiens","Determinaci�n de un ""core"" gen�tico de Mycobac..."
5904,1773.213,Mycobacterium tuberculosis strain VRFCWCF XDRT...,4369955.0,65.50000,99.5,100.0,0.0,1773,VRFCWCF XDRTB 232,2014-06-27T00:00:00Z,...,NaN,4316.0,4101.0,NaN,sputum,NaN,2010,India: Chennai,"Human, Homo sapiens",THIS STUDY AIMS TO EXPLORE THE DRUG RESISTANT ...


### 1.3 AMR selection and the most common genomes

In [7]:
dtypes = {
    'genome_id': str,
    'antibiotic': str,
    'resistant_phenotype': str,
    'measurement': str,
    'measurement_sign': str,
    'measurement_unit': str,
    'laboratory_typing_method': str,
    'laboratory_typing_method_version': str,
    'laboratory_typing_platform': str,
    'vendor': str,
    'testing_standard': str,
    'testing_standard_year': str,
    'source': str,
}

df_amr = pd.read_csv(
    PATRIC_AMR, 
    sep='\t',
    usecols=dtypes.keys(),
    dtype=dtypes
)

# Selected genomes
df_amr = pd.merge(df[['genome_id', 'genome_name']], df_amr, how='inner', on='genome_id')

# Phenotypes with R/S only
df_amr = df_amr[df_amr.resistant_phenotype.isin(['Resistant', 'Susceptible'])]

# Transform 'laboratory_typing_method_version' in to 'measurement' and 'measurement_unit'
convertions = {}
for t in df_amr[['measurement', 'measurement_unit', 'laboratory_typing_method_version']].itertuples():
    if t.laboratory_typing_method_version is not np.nan:
        m = t.laboratory_typing_method_version.split()
        # assert len(m) > 1, f"Not a measurement: {t}"
        if len(m) > 1:
            convertions[t.Index] = {'measurement': m[0], 'unit': m[1]}
for idx in convertions:
    df_amr.loc[idx, 'measurement'] = convertions[idx]['measurement']
    df_amr.loc[idx, 'measurement_unit'] = convertions[idx]['unit']
    
# Turn 'measurement' column dtype into valid number dtype
extract_number = lambda text: re.findall(r'[-+]?(?:\d*\.*\d)', text)[0]
df_amr['measurement'] = pd.to_numeric(df_amr.measurement.map(extract_number, na_action='ignore'))

# Remove sample without measurement
df_amr = df_amr.dropna(subset=['measurement'])

# Assign NA measurement sign with '='
df_amr['measurement_sign'] = df_amr['measurement_sign'].fillna('=')

# Use only sample in selected genomes
df_amr = pd.merge(df_amr, df['genome_id'], how='inner', on='genome_id')

In [8]:
# Plot the distributions
def phenotype_counts(df_amr):
    dist = defaultdict(int)
    for d in df_amr[['antibiotic', 'resistant_phenotype']].itertuples():
        name = f'{d.antibiotic}:{d.resistant_phenotype.lower()}'
        dist[name] += 1
    
    antibiotics = set([a.split(':')[0] for a in dist.keys()])
    phenotype_counts = [(dist[f'{a}:resistant'], dist[f'{a}:susceptible']) for a in antibiotics]
    return pd.DataFrame(phenotype_counts, index=list(antibiotics), columns=['resistant', 'susceptible'])


phenotype_counts(df_amr)

,resistant,susceptible
nicotinamide,77,74
ethambutol,280,600
moxifloxacin,90,197
rifampin,513,401
capreomycin,86,159
clofazimine,21,262
streptomycin,332,648
cycloserine,17,109
ofloxacin,186,337
ethionamide,188,210


In [9]:
selected_antibiotics = ['isoniazid', 'streptomycin', 'rifampin', 'ethambutol']

In [10]:
# Presences of the most common genomes in major antibiotics
def get_genomes(df_amr, antibiotic):
    return df_amr[df_amr.antibiotic.str.contains(antibiotic)].genome_id

g_isoniazid = get_genomes(df_amr, 'isoniazid')
assert len(g_isoniazid) == 983 + 622
g_streptomycin = get_genomes(df_amr, 'streptomycin')
assert len(g_streptomycin) == 332 + 648
g_rifampin = get_genomes(df_amr, 'rifampin')
assert len(g_rifampin) == 513 + 401
g_ethambutol = get_genomes(df_amr, 'ethambutol')
assert len(g_ethambutol) == 280 + 600

common_genomes = set(g_isoniazid)
common_genomes.intersection_update(g_streptomycin)
common_genomes.intersection_update(g_rifampin)
common_genomes.intersection_update(g_ethambutol)

len(common_genomes)

710

In [11]:
df_amr = df_amr[df_amr.genome_id.isin(common_genomes) & df_amr.antibiotic.isin(selected_antibiotics)]

phenotype_counts(df_amr)

,resistant,susceptible
streptomycin,271,616
isoniazid,634,589
ethambutol,277,590
rifampin,339,381


### 1.4 Clinical breakpoints

In [12]:
print(
    df_amr.laboratory_typing_method.unique(),
    df_amr.antibiotic.unique(),
    df_amr.resistant_phenotype.unique(),
    sep='\n\n'
)

['Agar proportion' 'Agar dilution' 'Broth dilution']

['ethambutol' 'isoniazid' 'rifampin' 'streptomycin']

['Resistant' 'Susceptible']


In [13]:
# Breakpoints/Critical concentrations tables
# https://www.sciencedirect.com/science/article/pii/S1198743X16305110#tbl2
# https://www.idexx.com/media/filer_public/b2/04/b2047e39-bbf3-48c1-8c26-076b7f850922/microbiology-guide-interpreting-mic.pdf
# 
# https://en.wikipedia.org/wiki/Middlebrook_7H9_Broth
# https://en.wikipedia.org/wiki/Middlebrook_7H10_Agar
breakpoints = {
    'ethambutol': {'7H10': 5.0, '7H9': 5.0},
    'isoniazid': {'7H10': 0.2, '7H9': 0.1},
    'rifampin': {'7H10': 1.0,'7H9': 1.0},
    'streptomycin': {'7H10': 2.0, '7H9': 1.0},
}

# Rules:
#   Isolates that cannot grow at ‘critical’ concentrations were then defined as susceptible, 
#   whereas those that can grow were considered resistant
# 
# S = Susceptible
# R = Resistant
# NR = Non-resistant when the phenotype is S but the its MIC is NOT < breakpoint
# NS = Non-susceptible when the phenotype is R but the its MIC is NOT >= breakpoint*2
def reannotate_phenotype(measurement, antibiotic, phenotype, method):
    def validate_by_beakpoint(antibiotic, media):
        if phenotype == 'Susceptible':
            if not measurement <= breakpoints[antibiotic][media]: 
                return 'NR'
            else:
                return 'S'
        if phenotype == 'Resistant':
            if not measurement >= 2*breakpoints[antibiotic][media]:
                return 'NS'
            else:
                return 'R'
    
    if method == 'Agar proportion' or method == 'Agar dilution':
        return validate_by_beakpoint(antibiotic, '7H10')
    if method == 'Broth dilution':
        return validate_by_beakpoint(antibiotic, '7H9')
    
    
assert reannotate_phenotype(2.0, 'ethambutol', 'Resistant', 'Agar proportion') == 'NS' 
assert reannotate_phenotype(4.0, 'streptomycin', 'Susceptible', 'Agar dilution') == 'NR' 
assert reannotate_phenotype(1.0, 'rifampin', 'Resistant', 'Agar dilution') == 'NS'
assert reannotate_phenotype(2.0, 'rifampin', 'Resistant', 'Agar dilution') == 'R'
assert reannotate_phenotype(1.0, 'rifampin', 'Susceptible', 'Agar dilution') == 'S'

In [14]:
df_amr['phenotype'] = df_amr.apply(
    lambda r: reannotate_phenotype(
        r['measurement'], 
        r['antibiotic'], 
        r['resistant_phenotype'], 
        r['laboratory_typing_method']
    ),
    axis=1
)

df_amr.value_counts('phenotype')

phenotype
NR    1299
NS    1054
S      877
R      467
dtype: int64

### 1.6 Only use valid samples (without NR and NS)

In [15]:
df_amr_valid = df_amr[~df_amr.phenotype.isin(['NS', 'NR'])]
used_genomes = df_amr_valid.genome_id.unique()

print(f'Genomes: {len(used_genomes)}')
print(f'Valid tests: {len(df_amr_valid)}')
phenotype_counts(df_amr_valid)

Genomes: 510
Valid tests: 1344


,resistant,susceptible
streptomycin,89,288
isoniazid,251,270
ethambutol,41,112
rifampin,86,207


### 1.7 Remove extreme inconsistence samples

In [16]:
df_amr_valid = df_amr_valid.drop_duplicates(subset=['genome_id', 'antibiotic', 'phenotype'])

print(f'Genomes: {len(df_amr_valid.genome_id.unique())}')
print(f'Valid tests: {len(df_amr_valid)}')
phenotype_counts(df_amr_valid)

Genomes: 510
Valid tests: 1243


,resistant,susceptible
streptomycin,89,277
isoniazid,251,184
ethambutol,41,112
rifampin,86,203


In [17]:
def report_inconsistency(df_amr_valid):
    inconsistence_samples = []

    for antibiotic in df_amr_valid.antibiotic.unique():
        checker = defaultdict(list)
    
        for r in df_amr_valid[df_amr_valid.antibiotic == antibiotic].itertuples():
            checker[r.genome_id].append(r.phenotype)
    
        print(f'\nInconsistency in {antibiotic}:')
        for gid in checker.keys():
            if len(checker[gid]) > 1:
                columns = ['genome_id', 'antibiotic', 'phenotype', 'measurement', 'laboratory_typing_method']
                invalid_sample = df_amr_valid.loc[(df_amr.genome_id == gid), columns]
                print(invalid_sample)
                inconsistence_samples.append(gid)
        print('________________________________________________')
            
            
report_inconsistency(df_amr_valid)


Inconsistency in rifampin:
________________________________________________

Inconsistency in streptomycin:
     genome_id    antibiotic phenotype  measurement laboratory_typing_method
7351  1773.203    ethambutol         S          0.1           Broth dilution
7355  1773.203     isoniazid         S          0.1           Broth dilution
7359  1773.203      rifampin         S          0.1           Broth dilution
7361  1773.203  streptomycin         S          0.1           Broth dilution
7363  1773.203  streptomycin         R          4.0            Agar dilution
________________________________________________

Inconsistency in ethambutol:
________________________________________________

Inconsistency in isoniazid:
      genome_id antibiotic phenotype  measurement laboratory_typing_method
3160  1324218.3  isoniazid         S          0.2            Agar dilution
3162  1324218.3  isoniazid         R          1.0            Agar dilution
      genome_id    antibiotic phenotype  measur

In [18]:
bad_samples = ['1773.203', '1324218.3', '1324235.3']

df_amr_valid = df_amr_valid[~df_amr_valid.genome_id.isin(bad_samples)]

report_inconsistency(df_amr_valid)

print(f'Genomes: {len(df_amr_valid.genome_id.unique())}')
print(f'Valid tests: {len(df_amr_valid)}')
phenotype_counts(df_amr_valid)


Inconsistency in rifampin:
________________________________________________

Inconsistency in streptomycin:
________________________________________________

Inconsistency in ethambutol:
________________________________________________

Inconsistency in isoniazid:
________________________________________________
Genomes: 507
Valid tests: 1233


,resistant,susceptible
streptomycin,88,275
isoniazid,249,181
ethambutol,41,111
rifampin,86,202


## 2. Results for QC (Mash)

**REMEBER** to add the genome reference!
- Genome ID: **83332.12**
- Genome Name: **Mycobacterium tuberculosis H37Rv**

### 2.1 Export to list and also embed the reference genome

In [19]:
reference_id = '83332.12'

In [20]:
FNA_LIST_TXT = path.join(SAMPLES_DIR, '_fna_before_mash.txt')
GFF_LIST_TXT = path.join(SAMPLES_DIR, '_gff_before_mash.txt')

f_fna = open(FNA_LIST_TXT, 'w')
f_gff = open(GFF_LIST_TXT, 'w')

genome_ids = [reference_id]
genome_ids.extend(used_genomes)

for gid in genome_ids:
    f_fna.write(f'{gid}.fna\n')
    f_gff.write(f'{gid}.PATRIC.gff\n')

f_fna.close()
f_gff.close()

### 2.2 Export to Excel

In [21]:
# Export AMR to Excel
wanted_columns = [
    'genome_id',
    'genome_name',
    'antibiotic',
    'phenotype',
    'measurement',
    'measurement_unit',
    'laboratory_typing_method',
]
AMR_XLSX = path.join(DATA_DIR, 'amr_before_mash.xlsx')
df_amr_valid[wanted_columns].to_excel(AMR_XLSX, index=False)

# Export Genomes to Excel
wanted_columns = ['genome_id',
    'genome_name',
    'strain',
    'taxon_id', 
    'genome_length',
    'gc_content',
    'fine_consistency', 
    'checkm_completeness', 
    'checkm_contamination',
    'completion_date',
    'publication',
    'bioproject_accession',
    'biosample_accession',
    'assembly_accession',
    'genbank_accessions',
    'refseq_accessions',
    'sequencing_centers',
    'sequencing_status',
    'sequencing_platform',
    'sequencing_depth',
    'assembly_method',
    'chromosomes',
    'contigs',
    'sequences',
    'patric_cds',
    'refseq_cds',
    'isolation_site',
    'isolation_source',
    'isolation_comments',
    'collection_date',
    'geographic_location',
    'host_name',
    'comments'
]
GENOMES_XLSX = path.join(DATA_DIR, 'genomes_before_mash.xlsx')
df = df[df.genome_id.isin(used_genomes)]
df[wanted_columns].to_excel(GENOMES_XLSX, index=False)